In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import sys
sys.version
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import pandas_profiling

In [ ]:
df= pd.read_csv('../input/mall-customer-cluster/Mall_Customers.csv')
df.head()

In [ ]:
x = df.iloc[:, [3, 4]].values.copy()

In [ ]:
# checking if the data has any null values 
df.isna().sum()

In [ ]:
# dropping the first columns as not necessary 
df = df.iloc[:,1:]

In [ ]:
# converting the gender in numerical column using get_dummies
gen_df = pd.get_dummies(df.Gender,drop_first=True)

In [ ]:
# Concatinig the two dataframes and dropping the gender column
data = pd.concat([df,gen_df],axis=1)
data.drop(columns='Gender',inplace=True)
data.rename(columns = {'Male':'Gender'},inplace =True)

In [ ]:
# Viualising the data set based on Gender
sns.pairplot(data,hue='Gender')

In [ ]:
from pandas import plotting
plt.rcParams['figure.figsize'] = (15, 10)


plotting.andrews_curves(data, "Gender")
plt.title('Andrew Curves for Gender', fontsize = 20)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 8)
sns.countplot(df['Age'], palette = 'hsv')
plt.title('Distribution of Age', fontsize = 20)
plt.show()

In [ ]:
x = df['Annual Income (k$)']
y = df['Age']
z = df['Spending Score (1-100)']

sns.lineplot(x, y, color = 'blue')
sns.lineplot(x, z, color = 'pink')
plt.title('Annual Income vs Age and Spending Score', fontsize = 20)
plt.show()

In [ ]:
# Removing the outliers
def outliers_capping(x):
    x=x.clip(upper=x.quantile(0.99),lower = x.quantile(0.01))
    return x
data = data.apply(lambda x : outliers_capping(x))
data.head()

In [ ]:
# Standardising the data 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(data)
data_scale=scaler.transform(data)

In [ ]:
# Applying PCA even if it is not required
from sklearn.decomposition import PCA
p = PCA(n_components =2)
p.fit(data_scale)
data_pca = pd.DataFrame(p.transform(data_scale))
data_pca.columns=['P1','P2']

In [ ]:
from sklearn.cluster import KMeans
cluster = 20
wcss = []
for i in range(1,cluster):
    kmeans=KMeans(n_clusters=i,init='k-means++',random_state=0)
    kmeans.fit(data_pca)
    wcss.append(kmeans.inertia_)

In [ ]:
# Plotting elbow method the get the range of the cluster required 
plt.plot(range(1,20),wcss)
plt.xlabel('number of clusters')
plt.ylabel('loss func')
plt.title('elbow method')

In [ ]:
# Now we have got the range of the clusters values we will select the optimal one using silhouette score
from sklearn.metrics import silhouette_score
n_range=[2,3,4,5,6,7]
silhouette_scores = []
for i in n_range:
    silhouette_scores.append(silhouette_score(data_pca,KMeans(n_clusters=i,init='k-means++',).fit_predict(data_pca)))
    
plt.bar(n_range,silhouette_scores)  

In [ ]:
# no of cluster  = 4
kmeans = KMeans(n_clusters=4,init='k-means++',random_state=0)
kmeans.fit(data_pca)
labels = kmeans.labels_
labels

In [ ]:
colour={}
colour[0]='r'
colour[1]='g'
colour[2]='y'
colour[3]='b'
cesl = [colour[label] for label in labels]
cesl
plt.figure(figsize=(9,9))
plt.scatter(data_pca['P1'],data_pca['P2'],c=cesl)

In [ ]:
clusters=pd.concat([data_pca, pd.DataFrame({'cluster':labels})], axis=1)
clusters.head()

# for cols in data:
#     g = sns.FacetGrid(clusters, col = 'cluster')
#     g.map(plt.hist, cols)

In [ ]:
import scipy.cluster.hierarchy as sch

dendrogram = sch.dendrogram(sch.linkage(x, method = 'ward'))
plt.title('Dendrogam', fontsize = 20)
plt.xlabel('Customers')
plt.ylabel('Ecuclidean Distance')
plt.show()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
km=KMeans(n_clusters=4,init='k-means',random_state=0)
hc = AgglomerativeClustering(n_clusters = 4, affinity = 'euclidean', linkage = 'ward')
y_hc = hc.fit_predict(x)

plt.scatter(x[y_hc == 0, 0], x[y_hc == 0, 1], s = 100, c = 'pink', label = 'miser')
plt.scatter(x[y_hc == 1, 0], x[y_hc == 1, 1], s = 100, c = 'yellow', label = 'general')
plt.scatter(x[y_hc == 2, 0], x[y_hc == 2, 1], s = 100, c = 'cyan', label = 'target')
plt.scatter(x[y_hc == 3, 0], x[y_hc == 3, 1], s = 100, c = 'magenta', label = 'spendthrift')
plt.scatter(x[y_hc == 4, 0], x[y_hc == 4, 1], s = 100, c = 'orange', label = 'careful')
# plt.scatter(km.cluster_centers_[:,0], km.cluster_centers_[:, 1], s = 50, c = 'blue' , label = 'centeroid')

plt.style.use('fivethirtyeight')
plt.title('Hierarchial Clustering', fontsize = 20)
plt.xlabel('Annual Income')
plt.ylabel('Spending Score')
plt.legend()
plt.grid()
plt.show()

In [ ]:
kmeans = KMeans(n_clusters = 4, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
ymeans = kmeans.fit_predict(x)

plt.rcParams['figure.figsize'] = (10, 10)
plt.title('Cluster of Ages', fontsize = 30)

plt.scatter(x[ymeans == 0, 0], x[ymeans == 0, 1], s = 100, c = 'pink', label = 'Usual Customers' )
plt.scatter(x[ymeans == 1, 0], x[ymeans == 1, 1], s = 100, c = 'orange', label = 'Priority Customers')
plt.scatter(x[ymeans == 2, 0], x[ymeans == 2, 1], s = 100, c = 'lightgreen', label = 'Target Customers(Young)')
plt.scatter(x[ymeans == 3, 0], x[ymeans == 3, 1], s = 100, c = 'red', label = 'Target Customers(Old)')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 50, c = 'black')

plt.style.use('fivethirtyeight')
plt.xlabel('Age')
plt.ylabel('Spending Score (1-100)')
plt.legend()
plt.grid()
plt.show()


According to my own intuition by looking at the above clustering plot between the age of the customers and their corresponding spending scores, I have aggregated them into 4 different categories namely Usual Customers, Priority Customers, Senior Citizen Target Customers, Young Target Customers. Then after getting the results we can accordingly make different marketing strategies and policies to optimize the spending scores of the customer in the Mall.